In [1]:
import numpy as np

In [6]:
pdb_filepath = 'polymer_AA_relaxed-3.pdb'
n_header_lines = 2
n_atoms = 1445
n_asp = 48
n_peg = 117

column_delimiters = [[7,11], [13,16], [30,38], [38,46], [46,54]]
#                     index   name      x         y       z coord

MW_elements = {
  'H': 1.0079,
  'C': 12.0107,
  'O': 15.9994,
  'N': 14.0067,
}

def pdb_coord_formatted(coord_array):
  a = ['{:.3f}'.format(j) for j in coord_array]
  b = ['{:>8}'.format(i) for i in a]
  formatted_coords = ''
  for i in b:
    formatted_coords += i
  return formatted_coords

with open(pdb_filepath, 'r') as f:
  data_string = f.read()

data_string = data_string.split('\n')[n_header_lines:n_header_lines+n_atoms]

data = []
for line in data_string:
  data_entry = []
  for column_delimiter in column_delimiters:
    data_entry += [line[column_delimiter[0]:column_delimiter[1]]]
  data_entry[0] = int(data_entry[0])
  data_entry[1] = data_entry[1][0]
  data_entry[2] = float(data_entry[2])
  data_entry[3] = float(data_entry[3])
  data_entry[4] = float(data_entry[4])
  data += [data_entry]

data = {i[0]:{'Element':i[1], 'Coordinates':[i[2], i[3], i[4]]} for i in data}
  
P5 = []
Qa = []
EO = []

for i in range(0, n_asp):
  P5 += [list(13*i + np.array([0, 1, 2, 3, 8, 12]))]
  Qa += [list(13*i + np.array([4, 5, 6, 7, 9, 10, 11]))]
  if i == 0: # some weird edge case for indices of 
    P5[i] = list(np.array(P5[i])+1)
    P5[i][-1] = 13*(n_asp-1) + 14 + 3*n_peg + 1
    Qa[i] = list(np.array(Qa[i])+1)
  if i == n_asp - 1: # additional OH exists at the aspartic acid's closed end 
    P5[i] += [13*i+13, 13*i+14]

peg_offset_main_atoms = 13*n_asp+1
peg_offset_hydrogens = 13*(n_asp-1) + 14 + 3*n_peg + 1
for i in range(0, n_peg):
  main_atoms = list(peg_offset_main_atoms + 3*i + np.array([1, 2, 3]))
  hydrogens = list(peg_offset_hydrogens + 4*i + np.array([1, 2, 3, 4]))
  EO += [main_atoms + hydrogens]

beads = [P5, Qa, EO]

for i in range(len(beads)):
  bead_list = beads[i]
  
  for j in range(len(bead_list)):
    bead_atoms = bead_list[j]
    MW_sum = 0
    summation = 0
    
    for atom in bead_atoms:
      element = data[atom]['Element']
      MW = MW_elements[element]
      MW_sum += MW
      #
      coordinates = data[atom]['Coordinates']
      summation += MW * np.array(coordinates)
      #
      r_com = summation / MW_sum
    
    beads[i][j] = r_com
  
P5 = beads[0]
Qa = beads[1]
EO = beads[2]

formatted_coords = ''
for i in range(len(P5)):
  a = pdb_coord_formatted(P5[::-1][i])
  b = pdb_coord_formatted(Qa[::-1][i])
  formatted_coords += f'{a}\n{b}\n'
  
for i in range(len(EO)):
  a = pdb_coord_formatted(EO[i])
  formatted_coords += f'{a}\n'
  
print(f'''The following coordinates are the COM coordinates and 
can be pasted in the PDB file to replace rows {column_delimiters[2][0]} to {column_delimiters[4][1]} 
of the pdb (take note of the leading whitespaces):\n''')
print(formatted_coords)

The following coordinates are the COM coordinates and 
can be pasted in the PDB file to replace rows 30 to 54 
of the pdb (take note of the leading whitespaces):

 138.599  -0.046 103.920
 136.699  -1.199 106.368
 134.841   0.194 101.847
 137.251   1.186  99.505
 132.389  -0.167  98.961
 130.797  -1.157 101.917
 128.947   0.189  97.393
 131.348   1.174  95.046
 126.491  -0.190  94.532
 124.906  -1.168  97.495
 123.070   0.204  92.971
 125.481   1.183  90.633
 120.628  -0.198  90.120
 119.049  -1.176  93.085
 117.221   0.212  88.561
 119.639   1.189  86.230
 114.787  -0.205  85.719
 113.215  -1.182  88.687
 111.390   0.217  84.164
 113.813   1.193  81.839
 108.962  -0.210  81.329
 107.396  -1.186  84.300
 105.570   0.221  79.777
 107.996   1.196  77.455
 103.145  -0.213  76.946
 101.583  -1.188  79.918
  99.757   0.223  75.395
 102.185   1.198  73.076
  97.334  -0.215  72.566
  95.775  -1.190  75.540
  93.949   0.224  71.017
  96.378   1.199  68.699
  91.527  -0.216  68.189
  89.969  -1